In [18]:
import pandas as pd  
import random
import requests
from time import sleep
from requests import get
from bs4 import BeautifulSoup as soup

import difflib
api_key = "cf92d1cf751a68bde1f3b3aa76b677be"

In [19]:
#Function that retun the highest revenue movies data frame. Input: year,

def moviesData(year):
    urlDiscover='https://api.themoviedb.org/3/discover/movie?api_key='
    searchkey='&primary_release_year='+str(year)+'&sort_by=revenue.desc'
    response = requests.get(urlDiscover + api_key + searchkey)
    revenueResult = response.json() 
    revenueResultLimit= revenueResult['results']

    df = pd.DataFrame(columns=['Release_Date','ID','Title','Profit','Runtime','Genres'])
    
    
    for film in revenueResultLimit:

        filmDetails = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key +'&language=en-US')
        filmDetails = filmDetails.json()
        profit=filmDetails['revenue']-filmDetails['budget']
        df.loc[len(df)]=[film['release_date'],film['id'],film['title'],profit, filmDetails['runtime'],genres(filmDetails)] # store title and revenue in our dataframe 

    return df

In [20]:
def cast(filmID):
    castList=[]
    film=requests.get("http://api.themoviedb.org/3/movie/"+str(filmID)+"/casts?"+'api_key='+api_key).json()
    for i in film['cast'][:10]:
        fullName=" ".join(i['name'].split()[:3])
        castList.append(fullName)
    return castList
    

In [21]:
cast(427641)

['Dwayne Johnson',
 'Naomie Harris',
 'Malin Åkerman',
 'Jeffrey Dean Morgan',
 'Jake Lacy',
 'Joe Manganiello',
 'Marley Shelton',
 'P. J. Byrne',
 'Demetrius Grosse',
 'Jack Quaid']

In [22]:
def genres(filmDetail):
    genresList=[]
    #for i in filmDetail['genres']:
    #    genresList.append(i['name'])
    return filmDetail['genres'][0]['name']

In [23]:
moviesData(2018).head()

,Release_Date,ID,Title,Profit,Runtime,Genres
0,2018-04-27,299536,Avengers: Infinity War,1746239637,149,Adventure
1,2018-02-16,284054,Black Panther,1146739107,134,Action
2,2018-06-22,351286,Jurassic World: Fallen Kingdom,1133459585,129,Action
3,2018-06-15,260513,Incredibles 2,1041891456,118,Action
4,2018-12-21,297802,Aquaman,983689193,144,Action


In [24]:
#url="https://www.imdb.com/list/ls058011111/?sort=list_order,asc&mode=detail&page=1&ref_=nmls_vm_dtl"
#page=html_soup.find_all(class_="lister-item-content")

nameList=pd.DataFrame({"Rank":[],"Name":[] })
for pageindex in [1,2,3,4,5,6,7,8,9,10]:
    url="https://www.imdb.com/list/ls058011111/?sort=list_order,asc&mode=detail&page="+str(pageindex)+"&ref_=nmls_vm_dtl"
    response = get(url)
    html_soup=soup(response.text, "html.parser")
    page=html_soup.find_all(class_="lister-item-content")
    for index, element in enumerate(page):
        nameList.loc[len(nameList)]=[index+1+(pageindex-1)*100,element.find_all('a')[0].text[:-1]]
    sleep(random.random())

In [25]:
nameList.head(5)

,Rank,Name
0,1.0,Robert De Niro
1,2.0,Jack Nicholson
2,3.0,Marlon Brando
3,4.0,Denzel Washington
4,5.0,Clark Gable


In [26]:
def castRank(castName):
    if castName in nameList["Name"]:
        return  (nameList[nameList["Name"]==castName])
    else :
        dfTemp=nameList.copy()
        ratioList=[]
        for i in nameList["Name"]:
            ratio=difflib.SequenceMatcher(None,castName,i).ratio()
            ratioList.append(ratio)
            
        dfTemp["Ratio"]=ratioList
        if max(ratioList) == 0:
            return (0,0)
        elif max(ratioList)>0.7:
            dfTemp2=dfTemp[dfTemp["Ratio"]==max(ratioList)].copy()
            return (dfTemp2.iloc[0]["Rank"],dfTemp2.iloc[0]["Name"])
        else:
            return (0,0)

In [27]:
df=moviesData(2015)

In [28]:
df.head()

,Release_Date,ID,Title,Profit,Runtime,Genres
0,2015-12-18,140607,Star Wars: The Force Awakens,1823223624,136,Action
1,2015-06-12,135397,Jurassic World,1521713208,124,Action
2,2015-04-03,168259,Furious 7,1316249360,137,Action
3,2015-05-01,99861,Avengers: Age of Ultron,1125403694,141,Action
4,2015-07-10,211672,Minions,1082730962,91,Family


In [29]:
def castRankList(id):
    listFoo=pd.DataFrame({'Rank':[],'Name':[]})
    for name in cast(id):
        if castRank(name)[0]==0:
            pass
        else:
            listFoo.loc[len(listFoo)]=[castRank(name)[0],castRank(name)[1]]
    return listFoo.sort_values('Rank')

In [30]:
castRankList(420818)

,Rank,Name
0,170.0,Chiwetel Ejiofor
2,462.0,Seth Rogen
1,991.0,John Krasinski


In [31]:
castRank("JACK NICHOLSON")

(0, 0)

In [32]:
castRankList(299534)

,Rank,Name
6,73.0,Don Cheadle
0,125.0,Robert Downey Jr.
8,274.0,Benedict Cumberbatch
4,283.0,Scarlett Johansson
3,315.0,Chris Hemsworth
5,454.0,Jeremy Renner
1,457.0,Chris Evans
7,465.0,Paul Rudd
2,516.0,Mark Ruffalo


In [33]:
def double(x):
    return 2*x
    

In [34]:
double(2)

4

In [35]:
df2019=moviesData(2019)

KeyError: 'revenue'

In [ ]:
import sqlite3 as db


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(df2019["Title"],df2019["Profit"])
plt.xticks(df2019["Title"], rotation=90)
plt.show()

In [2]:
ls

MovieDataBase.ipynb          actorRank.db
MovieDataBase.ipynb (1).zip  moviesData
MovieDataBase.ipynb.zip      premoviedb.ipynb


In [3]:
df = pd.read_csv('moviesData')

In [7]:
pplot.df

NameError: name 'pplot' is not defined